In [2]:
import folium
from geopy.distance import geodesic

# Dados de 25 bairros de Salvador com suas coordenadas
bairros = [
    {"nome": "Barra", "location": [-13.0134, -38.5087]},
    {"nome": "Pelourinho", "location": [-12.9714, -38.5109]},
    {"nome": "Rio Vermelho", "location": [-13.0161, -38.4906]},
    {"nome": "Pituba", "location": [-12.9994, -38.4602]},
    {"nome": "Itapuã", "location": [-12.9519, -38.3538]},
    {"nome": "Ondina", "location": [-13.0092, -38.5002]},
    {"nome": "Graça", "location": [-13.0021, -38.5174]},
    {"nome": "Brotas", "location": [-12.9847, -38.4850]},
    {"nome": "Amaralina", "location": [-12.9981, -38.4609]},
    {"nome": "Stiep", "location": [-12.9989, -38.4515]},
    {"nome": "Caminho das Árvores", "location": [-12.9801, -38.4594]},
    {"nome": "Imbuí", "location": [-12.9896, -38.4386]},
    {"nome": "Paralela", "location": [-12.9573, -38.4373]},
    {"nome": "Boca do Rio", "location": [-12.9713, -38.4400]},
    {"nome": "Patamares", "location": [-12.9275, -38.3822]},
    {"nome": "Piatã", "location": [-12.9351, -38.3780]},
    {"nome": "Stella Maris", "location": [-12.9103, -38.3348]},
    {"nome": "Centro", "location": [-12.9711, -38.5014]},
    {"nome": "Comércio", "location": [-12.9664, -38.5130]},
    {"nome": "São Cristóvão", "location": [-12.8997, -38.3536]},
    {"nome": "Alphaville", "location": [-12.9398, -38.3736]},
    {"nome": "Pernambués", "location": [-12.9843, -38.4608]},
    {"nome": "Iguatemi", "location": [-12.9796, -38.4551]},
    {"nome": "Federação", "location": [-13.0048, -38.5143]},
    {"nome": "Bonfim", "location": [-12.9262, -38.5107]},
]

# Calcular as distâncias entre os bairros e criar arestas
arestas = []
for i in range(len(bairros)):
    for j in range(i + 1, len(bairros)):
        distancia = geodesic(bairros[i]["location"], bairros[j]["location"]).meters
        arestas.append({"from": i, "to": j, "peso": distancia})

# Cria um mapa centrado em Salvador
mapa = folium.Map(location=[-12.9714, -38.5014], zoom_start=12)

for aresta in arestas:
    inicio = bairros[aresta["from"]]["location"]
    fim = bairros[aresta["to"]]["location"]
    nome_inicio = bairros[aresta["from"]]["nome"]
    nome_fim = bairros[aresta["to"]]["nome"]
    folium.PolyLine(
        locations=[inicio, fim],
        weight=2,
        color='blue',
        tooltip=f"{nome_inicio} ↔ {nome_fim}: {aresta['peso']:.2f} m"
    ).add_to(mapa)

# Adiciona os nós (bairros)
for bairro in bairros:
    folium.Marker(
        location=bairro["location"],
        tooltip=bairro["nome"],
        popup=bairro["nome"],
        icon = folium.Icon(color="blue"),
    ).add_to(mapa)

# Exibe o mapa
mapa.save("mapa_bairros_salvador.html")

mapa

In [7]:
import json
from geopy.distance import geodesic
import igraph as ig

source, target = "Barra", "Stiep"
with open('bairros.json', 'r') as json_bairros:
    locacoes = json.load(json_bairros)

# Criando o grafo igraph
grafo = ig.Graph(directed=True)

# Adicionando vértices com atributos de localização
for nome, localizacao in locacoes.items():
    grafo.add_vertex(name=nome, location=localizacao)

# Adicionando arestas com pesos calculados usando geodesic distance (em km)
vertices = grafo.vs['name']
for origem in vertices:
    for destino in vertices:
        if origem != destino:
            origem_loc = locacoes[origem]
            destino_loc = locacoes[destino]
            distancia = geodesic(origem_loc, destino_loc).kilometers
            grafo.add_edge(origem, destino, weight=distancia)

# Encontrando o caminho mais curto com Dijkstra
caminho = grafo.get_shortest_paths(source, to=target, weights='weight', output='vpath')[0]
distancia_total = grafo.shortest_paths_dijkstra(source, target, weights='weight')[0][0]

# Obtendo os nomes dos bairros no caminho
bairros_no_caminho = [grafo.vs[vertice]['name'] for vertice in caminho]

caminho

/tmp/ipykernel_1218766/2369267995.py:28: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  distancia_total = grafo.shortest_paths_dijkstra(source, target, weights='weight')[0][0]


[0, 9]